In [1]:
# Importando as bibliotecas
import pandas as pd
from string import ascii_uppercase
import os
import pg8000

######################################## execução da extração do corretor com ajustes para integração ########################################
# ajutando para que o IDENT do corretor traga as informações de escola, serie, letra da turma e numero do aluno
def processar_ident(ident):
    ident_str = str(ident).zfill(6)
    
    # Converter para inteiros com tratamento de segurança
    tdc_escola = int(ident_str[:2]) if ident_str[:2].isdigit() else 0
    serie = int(ident_str[2]) if ident_str[2].isdigit() else 0
    letra = ascii_uppercase[int(ident_str[3])-1] if ident_str[3].isdigit() and 1 <= int(ident_str[3]) <= 26 else 'a'
    numero = int(ident_str[4:]) if ident_str[4:].isdigit() else 0
    
    return tdc_escola, serie, letra, numero

# processamento do arquivo
def parse_arquivo(conteudo):
    blocos = conteudo.split('--------------------------') # identificador de final de bloco
    dados = []
    
    for bloco in blocos:
        linhas = [linha.strip() for linha in bloco.split('\n') if linha.strip()] # Ajuste para quebra de linha
        
        ident = next((linha.split(': ')[1] for linha in linhas if linha.startswith('IDENT.:')), None) # receber dados até ident
        if not ident or len(ident) != 6:
            continue

        # identificação de início de alternativas para gerar repetição 
        respostas_linhas = []
        for i, linha in enumerate(linhas):
            if 'Respostas (+ indica resposta correta):' in linha:
                respostas_linhas = linhas[i+1:]
                break

        # processando ident após o recebimento
        tdc_escola, serie, letra, numero = processar_ident(ident)

        # obter número da alternativa e reposta
        for linha in respostas_linhas:
            if '.' in linha:
                partes = linha.split('.', 1)
                questao = int(partes[0].strip())

                 # tratamento de respostas
                resposta = partes[1].strip().split()[0]

                # Converter respostas inválidas para "Em Branco"
                if 'Resposta inválida' in linha:
                    resposta = 'Em Branco'
                else:
                    # Remover marcador para utilizar apenas as letras das alternativas
                    resposta = resposta.replace('+', '').strip()
                
                dados.append([
                    f"{ident}{questao:02d}",  # Formato ID com zero padding
                    int(ident),
                    tdc_escola,
                    serie,
                    letra,
                    numero,
                    questao,
                    resposta
                ])
    
    return pd.DataFrame(dados, columns=[
        'ID', 'ID_PROVA', 'TDC_ESCOLA_ID', 'NUMERO_SERIE',
        'LETRA_TURMA', 'NUMERO_ALUNO_TURMA', 'QUESTAO', 'RESPOSTA'
    ])

# Configurar caminho da pasata local
pasta = r'.\resultados escolas'
df_corretor = pd.DataFrame()

# Processar todos os arquivos
for arquivo in os.listdir(pasta):
    if arquivo.lower().endswith('.txt'):
        caminho_completo = os.path.join(pasta, arquivo)
        
        with open(caminho_completo, 'r', encoding='utf-8') as f:
            conteudo = f.read()
            df = parse_arquivo(conteudo)
            df_corretor = pd.concat([df_corretor, df])

######################################## execuçãoda extração das dados dos alunos com ajustes para integração ########################################
# Configurar conexão com banco local
try:
    conn = pg8000.connect(
        host="localhost",
        database="educacao",
        user="postgres",
        password="admin",
        port=5432
    )

    # Query para extrair dados da view
    query = "SELECT * FROM educacao.view_aluno_turma_detalhada"

    # Executar a query e criar DataFrame
    cursor = conn.cursor()
    cursor.execute(query)
    dados_aluno = cursor.fetchall()
    col_names = [desc[0] for desc in cursor.description]
    
    df_dados_aluno = pd.DataFrame(dados_aluno, columns=col_names)
    
finally:
    if 'conn' in locals():
        conn.close()

######################################## execuçãoda extração das habilidades com ajustes para integração ########################################
# set de Google Sheets ID e o nome da planilha
sheet_id = "1Q6UcMBiSNvPj09Xj-mcpVq7_EslYBxAB6gmHonDtdwI"
sheet_name = "Prova"

# Criando URL para importar
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Lendo o dado em um dataframe
df_habilidades = pd.read_csv(url, skiprows=2)

# Ajustando o cabeçalho do DataFrame
df_habilidades.columns = ['id', 'questao', 'objetivo', 'alternativa', 'descricao', 'tipo']

#Ajustando tipo da coluna
df_habilidades['questao'] = df_habilidades['questao'].astype(int)

######################################## Integrando os dataframes após extract e transform ########################################
# Merge 1: Dados do corretor + dados dos alunos
merged_aluno = pd.merge(
    df_corretor,
    df_dados_aluno,
    left_on=['TDC_ESCOLA_ID', 'NUMERO_SERIE', 'LETRA_TURMA', 'NUMERO_ALUNO_TURMA'],
    right_on=['tdc_escola_id', 'numero_serie', 'letra_turma', 'numero_aluno_turma'],
    how='left'
)

#Merge 2: Merge 1 + habilidades
df_integrado = pd.merge(
    merged_aluno,
    df_habilidades,
    left_on=['QUESTAO', 'RESPOSTA'],
    right_on=['questao', 'alternativa'],
    how='left'
)

# Remover colunas duplicadas e ajustar nomes
df_integrado = df_integrado.drop(columns=[
    'tdc_escola_id', 'numero_serie', 'letra_turma', 'numero_aluno_turma',
    'questao', 'alternativa',
    'id_x', 'id_y'
])

# # Visualizando DataFrame Integrado
df_integrado

,ID,ID_PROVA,TDC_ESCOLA_ID,NUMERO_SERIE,LETRA_TURMA,NUMERO_ALUNO_TURMA,QUESTAO,RESPOSTA,ano_letivo,escola,serie,objetivo,descricao,tipo
0,05610101,56101,5,6,A,1,1,A,2025,EMEF DIVAIL SANTOS,SEXTO ANO,Ordem de resolução númerica,compreendeu,correta
1,05610102,56101,5,6,A,1,2,E,2025,EMEF DIVAIL SANTOS,SEXTO ANO,Ordem de resolução númerica,compreendeu,correta
2,05610103,56101,5,6,A,1,3,A,2025,EMEF DIVAIL SANTOS,SEXTO ANO,Ordem de resolução númerica,compreendeu,correta
3,05610104,56101,5,6,A,1,4,D,2025,EMEF DIVAIL SANTOS,SEXTO ANO,Verificar horas em ponteiro,compreendeu,correta
4,05610105,56101,5,6,A,1,5,E,2025,EMEF DIVAIL SANTOS,SEXTO ANO,Verificar horas em ponteiro,compreendeu,correta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4385,02633006,26330,2,6,C,30,6,C,2025,EMEF PEDRO BONIFACIO,SEXTO ANO,Verificar horas em ponteiro,compreendeu,correta
4386,02633007,26330,2,6,C,30,7,A,2025,EMEF PEDRO BONIFACIO,SEXTO ANO,Grandezas e Medidas,compreendeu,correta
4387,02633008,26330,2,6,C,30,8,B,2025,EMEF PEDRO BONIFACIO,SEXTO ANO,Grandezas e Medidas,compreendeu,correta
4388,02633009,26330,2,6,C,30,9,A,2025,EMEF PEDRO BONIFACIO,SEXTO ANO,Grandezas e Medidas,Verificar diferença entre perímetro e área,distrator
